In [2]:
import asyncio
from playwright.async_api import async_playwright
import qrcode
from PIL import Image
import io
import base64
from pyzbar import pyzbar
import os

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

In [4]:
await page.goto("https://web.whatsapp.com")

<Response url='https://web.whatsapp.com/' request=<Request url='https://web.whatsapp.com/' method='GET'>>

In [6]:
async def get_list_of_messages():
    """get_list_of_messages()

    gets the list of messages in the page
    """
    messages = await page.query_selector_all(
        '//*[@id="pane-side"]/div[2]/div/div/child::div'
    )

    clean_messages = []
    for message in messages:
        _message = await message.inner_text()  # Use inner_text to get the text content
        _message = _message.split("\n")
        if len(_message) == 2:
            clean_messages.append(
                {
                    "sender": _message[0],
                    "time": _message[1],
                    "message": "",
                    "unread": False,
                    "no_of_unread": 0,
                    "group": False,
                }
            )
        elif len(_message) == 3:
            clean_messages.append(
                {
                    "sender": _message[0],
                    "time": _message[1],
                    "message": _message[2],
                    "unread": False,
                    "no_of_unread": 0,
                    "group": False,
                }
            )
        elif len(_message) == 4:
            clean_messages.append(
                {
                    "sender": _message[0],
                    "time": _message[1],
                    "message": _message[2],
                    "unread": _message[-1].isdigit(),
                    "no_of_unread": int(_message[-1]) if _message[-1].isdigit() else 0,
                    "group": False,
                }
            )
        elif len(_message) == 5:
            clean_messages.append(
                {
                    "sender": _message[0],
                    "time": _message[1],
                    "message": "",
                    "unread": _message[-1].isdigit(),
                    "no_of_unread": int(_message[-1]) if _message[-1].isdigit() else 0,
                    "group": True,
                }
            )
        elif len(_message) == 6:
            clean_messages.append(
                {
                    "sender": _message[0],
                    "time": _message[1],
                    "message": _message[4],
                    "unread": _message[-1].isdigit(),
                    "no_of_unread": int(_message[-1]) if _message[-1].isdigit() else 0,
                    "group": True,
                }
            )
        else:
            print(f"Unknown message format: {_message}")
    return clean_messages


await get_list_of_messages()

Unknown message format: ['Pappa']
Unknown message format: ['TASC-Deepshika Poojary']
Unknown message format: ['slaves of rakuten']
Unknown message format: ['Mom']
Unknown message format: ['Shaun Orwil Fernandes']
Unknown message format: ['Sanju Akka']
Unknown message format: ["AIML BO'25"]


[{'sender': 'Zih',
  'time': '9:30 PM',
  'message': 'why',
  'unread': False,
  'no_of_unread': 0,
  'group': False},
 {'sender': 'SHINE FAMILY',
  'time': '47',
  'message': '',
  'unread': False,
  'no_of_unread': 0,
  'group': False},
 {'sender': 'AIML Official grp 2021-25',
  'time': '1:39 PM',
  'message': '',
  'unread': False,
  'no_of_unread': 0,
  'group': True},
 {'sender': 'Asheesh',
  'time': '3:46 PM',
  'message': 'If not atleast one',
  'unread': False,
  'no_of_unread': 0,
  'group': False},
 {'sender': 'Dsu dev hack',
  'time': '5:08 PM',
  'message': '',
  'unread': False,
  'no_of_unread': 0,
  'group': True},
 {'sender': 'PEC - I Cloud computing',
  'time': '7:59 PM',
  'message': 'https://meet.google.com/goz-xzhz-ghk (1st hour) https://meet.google.com/fqc-cfdk-duz (2nd hour)',
  'unread': True,
  'no_of_unread': 3,
  'group': True},
 {'sender': 'Batch 3...',
  'time': '9:06 PM',
  'message': 'Tell yes guys',
  'unread': True,
  'no_of_unread': 2,
  'group': True},

In [24]:
async def search_person(name: str, message: str):
    search_input_selector = (
        "div[contenteditable='true'][role='textbox'][data-lexical-editor='true']"
    )
    await page.wait_for_selector(search_input_selector, timeout=60000)
    search_input = await page.query_selector(search_input_selector)

    if search_input:
        # Type the name into the search input
        await search_input.click()
        await search_input.type(
            name, delay=100
        )  # Adding a slight delay can help with typing simulation
        await page.keyboard.press(
            "Enter"
        )  # Use page.keyboard instead of search_input.keyboard
        # Wait for search results to load
        await page.wait_for_timeout(2000)

        # Wait for the specific search result to be visible and click on it
        # search_result_selector = "p.selectable-text[title='kausthubh']"
        # search_result = await wa.page.wait_for_selector(
        #     search_result_selector, timeout=60000
        # )

        # if search_result:
        #     await search_result.click()
        #     await page.keyboard.press("Enter")  # Use page.keyboard instead of search_result.press

        #     print(f"Chat with '{name}' opened.")

    await page.fill(message)  # Use the 'message' parameter instead of hardcoded text
    await page.keyboard.press("Enter")

In [26]:
await search_person("kausthubh", "this is a test message pls ignore")

TypeError: Page.type() missing 1 required positional argument: 'text'

In [14]:
while True:
    m = await get_list_of_messages()

    if m[0]["unread"] == True:
        await search_person(
            m[0]["sender"],
            "This is a test message for my hackathon project please ignore",
        )
        break

TypeError: Page.type() missing 1 required positional argument: 'text'

In [ ]:
def decode_qr_from_image(image_bytes):
    try:
        image = Image.open(io.BytesIO(image_bytes))
        decoded_objects = pyzbar.decode(image)
        for obj in decoded_objects:
            return obj.data.decode("utf-8")
        return None
    except Exception as e:
        print(f"Failed to decode QR from image: {e}")
        return None


def display_qr_in_terminal(data):
    try:
        qr = qrcode.QRCode()
        qr.add_data(data)
        qr.make()
        qr.print_ascii(invert=True)
    except Exception as e:
        print(f"Failed to display QR in terminal: {e}")

In [ ]:
qrinfo = None

try:
    await page.wait_for_selector("div._akau", timeout=600)
    print("QR code container detected.")

    qr_div = await page.query_selector("div._akau")
    if qr_div:
        qr_info = await qr_div.get_attribute("data-ref")
        if qr_info:
            qrinfo = qr_info
    else:
        print("data-ref attribute not found. Proceeding to screenshot method.")
except Exception as e:
    print(f"Error while extracting data-ref: {e}")

In [ ]:
qrinfo

In [ ]:
display_qr_in_terminal(qrinfo)

In [ ]:
screenshot = await page.screenshot()
qr_info = decode_qr_from_image(screenshot)
if qr_info:
    return qr_info
else:
    print("Failed to decode QR code from screenshot.")
    return None

In [ ]:
async def extract_login_qr(self):
    assert page != None

In [ ]:
try:
    # Wait for the search input to be visible using a more stable selector
    search_input_selector = (
        "div[contenteditable='true'][role='textbox'][data-lexical-editor='true']"
    )
    await page.wait_for_selector(search_input_selector, timeout=60000)
    search_input = await page.query_selector(search_input_selector)

    if search_input:
        # Type the name 'kausthubh' into the search input
        await search_input.click()
        await search_input.type(
            "kausthubh", delay=100
        )  # Adding a slight delay can help with typing simulation

        # Wait for search results to load
        await page.wait_for_timeout(2000)

        # Wait for the specific search result to be visible and click on it
        search_result_selector = "p.selectable-text[title='kausthubh']"
        search_result = await page.wait_for_selector(
            search_result_selector, timeout=60000
        )

        if search_result:
            await search_result.click()
            print("Chat with 'kausthubh' opened.")
        else:
            print(
                "Search result for 'kausthubh' not found. Clicking on the first chat."
            )

            # Selector for the first chat in the chat list
            first_chat_selector = (
                "div.x1y332i5:nth-child(3)"  # Update this selector based on actual DOM
            )
            first_chat = await page.wait_for_selector(
                first_chat_selector, timeout=60000
            )

            if first_chat:
                await first_chat.click()
                print("First chat clicked.")
            else:
                print("Failed to find the first chat.")
    else:
        print("Search input not found.")
except Exception as e:
    print(f"Error while searching for 'kausthubh': {e}")

## Closing the browser and saving the context

In [ ]:
await context.storage_state(path="browser_context.json")

if browser:
    await browser.close()
elif context:
    await context.close()

await playwright.stop()

In [ ]:
page2 = await browser.new_page()